In [ ]:
from scipy.special import softmax
import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support

In [ ]:
def read_data(filename):
    PATH  = "/home/dona/bm75/Project_Chathurika/Classification/data/data/remove_background/results/"
    #PATH  = "/home/dona/bm75/Project_Chathurika/Classification/data/results/"
    f = open(PATH + filename, 'r')
    f.readline()
    
    predictions = []
    labels = []
    for line in f:
        splits = line.strip().split(']')
        #print(splits[-1])
        labels.append(int(splits[-1].split(',')[-1]))
        if len(splits[0].split()) > 2:
            preds = [float(splits[0].split()[1]), float(splits[0].split()[-1])] 
        else:
            preds = [float(splits[0].split()[0].split('[')[-1]), float(splits[0].split()[-1])]
        #print(preds, softmax(np.array(preds))[1])
        predictions.append(softmax(np.array(preds))[1])
    # print("=============================")
    return labels, predictions

#filenames = ['abdomen']
filenames = ['body', 'head', 'thorax', 'abdomen']
# filenames = ['body_resnet101']
train_labels_list = []
test_labels_list = []
train_predictions_list = []
test_predictions_list = []
for filename in filenames:
    labels, predictions = read_data(filename + '_train.txt')
    predictions = np.array(predictions).reshape(-1,1)
    train_labels_list.append(labels)
    train_predictions_list.append(predictions)
    
    labels, predictions = read_data(filename + '_test.txt')
    
    predictions = np.array(predictions).reshape(-1,1)

    print(filename)
    predicted_labels = predictions.copy()
    predicted_labels[predicted_labels>0.5]=1
    predicted_labels[predicted_labels<=0.5]=0
    print(accuracy_score(labels, predicted_labels))
    p, r, f, _ = precision_recall_fscore_support(labels, predicted_labels, average="binary")
    print(p, r, f)
    test_labels_list.append(labels)
    test_predictions_list.append(predictions)

In [ ]:
from scipy.stats import pearsonr
for i in range(4):
    for j in range(4):
        if  i == j:
             continue
        print(i,j)
        print(pearsonr(train_predictions_list[i].reshape(-1,), train_predictions_list[j].reshape(-1,1)))

In [ ]:
from sklearn.tree import DecisionTreeClassifier

train_X = np.concatenate(train_predictions_list[:2] + train_predictions_list[2:], axis = -1)
test_X = np.concatenate(test_predictions_list[:2] + test_predictions_list[2:], axis = -1)

model = LogisticRegression().fit(train_X, train_labels_list[0])
#model = DecisionTreeClassifier().fit(train_X, train_labels_list[0])

print(train_X)
ensemble_train_preds = model.predict(train_X)
ensemble_test_preds = model.predict(test_X)

# ensemble_test_preds = np.round(np.mean(np.array(test_predictions_list).astype(float), axis =0))
# ensemble_test_preds

In [ ]:
print(accuracy_score(train_labels_list[0], ensemble_train_preds))
print(accuracy_score(test_labels_list[0], ensemble_test_preds))
print(precision_recall_fscore_support(test_labels_list[0], ensemble_test_preds, average="binary"))

In [ ]:
import pickle
file = "/home/dona/bm75/Project_Chathurika/Classification/data/data/remove_background/" + 'ensembling_model.sav'
pickle.dump(model, open(file, 'wb'))
loaded_model = pickle.load(open(file, 'rb'))
result = loaded_model.predict(train_X)
print(len(result))
print(len(train_labels_list[0]))

In [ ]:
objects = ('body', 'head', 'thorax', 'abdomen')
y_pos = np.arange(len(objects))
print(model.coef_)

plt.bar(y_pos, model.coef_[0], align='center', alpha=0.5)
plt.xticks(y_pos, objects)
plt.show()